# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
import torch
from torch import nn,optim
import random
import re
device=torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=2)

# LSTM Model -- final model

In [3]:
def encode(word,stoi=None):
    l=[]
    if stoi is not None:
        for i in word:
            l.append(stoi[i])
        return l
    for i in word:
        l.append(ord(i)-97)
    return l
def decode(tokens,itos):
    l=[]
    for i in tokens:
        l.append(itos[i])
    return "".join(l)
class TextDataset(torch.utils.data.Dataset):
    def __init__(self,text_file="words_250000_train.txt",start_tok="<sos>",end_tok="<eos>"):
        super(TextDataset,self).__init__()
        self.data=open(text_file).readlines()
        self.data=[ re.sub("\n","",i) for i in self.data] #i[:-1] to remove the last '\n'
        self.vocab=list(set("".join(self.data)))
        self.vocab.sort()
        self.itos={i:ch for i,ch in enumerate(self.vocab)}
        letters=[ch for ch in self.vocab]
        self.vocab.append("_") 
        self.vocab.append("<pad>")
        self.vocab.append(start_tok)
        if end_tok not in self.vocab:
            self.vocab.append(end_tok)
        self.vocab.sort()
        self.vocab_size=len(self.vocab)
        self.stoi={ch:i for i,ch in enumerate(self.vocab)}
        self.X_data=[]
        self.Y_data=[]
        for word in self.data:
            chars=list(set(word))
            random.shuffle(chars)
            temp=[]
            for j in chars:
                word=re.sub(j,"_",word)
                #left_out=[ch for ch in letters if ch not in word]
                left_out=[]
                for ch in letters:
                    if ch not in word:
                        left_out.append(ch)
                    else:
                        left_out.append("<pad>")
                tempx=[self.stoi[start_tok],]+encode(word,self.stoi)+[self.stoi[end_tok],]
                tempy=torch.tensor(encode(j))
                temp.append(tempy)
                for i in temp:
                    num=random.sample(list(range(5)),1)[0]
                    keep_out=random.sample(left_out,min(num,len(left_out)))
                    left_out=[ch for ch in left_out if ch not in keep_out]
                    remains=[]
                    for ch in left_out:
                        if ch=="<pad>" or ch not in keep_out:
                            remains.append(ch)
                        elif ch in keep_out:
                            remains.append("<pad>")
                    self.X_data.append(torch.tensor(tempx+encode(remains,self.stoi)))
                    self.Y_data.append(i)
    def __len__(self):
        return len(self.X_data)
    def __getitem__(self,idx):
        return self.X_data[idx],self.Y_data[idx]
    
def MakeBatch(X_data,Y_data,batch_size=None):
    if batch_size is not None:
        idx=random.sample(range(len(X_data)),batch_size)
        return [X_data[i] for i in idx],torch.tensor([Y_data[i] for i in idx])
    else:
        return X_data,torch.tensor([Y_data])
    
class Network(nn.Module):
    def  __init__(self,vocab_size,pad_idx,embedding_dim=300,hidden_dim=50,num_layers=2,bidirectional=False,lin_layers=[],activation=nn.ReLU(),outputs=26)-> None:
        super(Network,self).__init__()
        self.pad_idx=pad_idx
        self.num_layers=num_layers
        self.hidden_dim=hidden_dim
        self.bidirectional=bidirectional
        self.Embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=self.pad_idx)
        self._init_weights(self.Embedding)
        self.LSTM=nn.LSTM(embedding_dim,hidden_dim,num_layers,bidirectional=bidirectional,dropout=0.4,batch_first=True)
        for layer in range(num_layers):
            for weight in self.LSTM._all_weights[layer]:
                if "weight" in weight:
                    nn.init.xavier_uniform_(getattr(self.LSTM,weight))
                if "bias" in weight:
                    nn.init.uniform_(getattr(self.LSTM,weight))
        self.lin_layers=[]
        
        if(bidirectional):
            #self.ln_norm=nn.LayerNorm(2*hidden_dim*num_layers)
            self.convert=nn.Linear(2*hidden_dim*num_layers,lin_layers[0])
        else:
            #self.ln_norm=nn.LayerNorm(hidden_dim*num_layers)
            self.convert=nn.Linear(hidden_dim*num_layers,lin_layers[0])
        for i,j in zip(lin_layers,lin_layers[1:]):
            self.lin_layers.append(nn.Linear(i,j))
        self.lin_layers=nn.ModuleList(self.lin_layers)
        self.outputs=nn.Linear(lin_layers[-1],outputs)
        self._init_weights(self.outputs)
        self.act=activation
        #self.hidden = self.init_hidden(batch_size)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight.data)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            nn.init.kaiming_normal_(module.weight.data)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
    
    def forward(self,x,y=None):
        x=torch.nn.utils.rnn.pad_sequence(x,batch_first=True,padding_value=self.pad_idx)
        x=self.Embedding(x)
        o,(h,c)=self.LSTM(x,self.init_hidden(x.shape[0]))
        h=h.permute(1,0,2)
        x=h.reshape(h.shape[0],h.shape[1]*h.shape[2])
        x=self.act(self.convert(x))
        for layer in self.lin_layers:
            x=layer(x)
            x=self.act(x)
        x=self.outputs(x)
        if y is None:
            loss=None
        else:
            loss=torch.nn.functional.cross_entropy(x,y)
        return x,loss
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.num_layers*2 if self.bidirectional else self.num_layers,batch_size,self.hidden_dim).to(device)
        cell_state = torch.zeros(self.num_layers*2 if self.bidirectional else self.num_layers,batch_size,self.hidden_dim).to(device)
        return (hidden_state, cell_state)
    
    def generate(self,x):
        logits,_=self(x)
        probs=torch.nn.functional.softmax(logits,dim=-1)
        idx=torch.multinomial(probs.cpu().detach(),1).item()
        return idx

In [4]:
#hyperparameters
embedding_size=150
hidden_dim=300
num_layers=3
bidirectional=True
lin_layers=[150,100]
activation=nn.SELU()
outputs=26
epochs=10000
lr=0.001
batchSize=3000

# Need not run the below cells, these were used for training purposes. Model weights have been provided in the zip file

In [126]:
Train_data=TextDataset("words_250000_train.txt","<sos>","<eos>")

In [127]:
idx=list(range(len(Train_data.X_data)))
random.shuffle(idx)
Train_data.X_data=[Train_data.X_data[i] for i in idx]
Train_data.Y_data=[Train_data.Y_data[i] for i in idx]
n=int(0.95*len(idx))
TrainX,TrainY=Train_data.X_data[:n],Train_data.Y_data[:n]
ValX,ValY=Train_data.X_data[n:],Train_data.Y_data[n:]
n

7132281

In [128]:
n<batchSize*epochs

True

In [129]:
model=Network(Train_data.vocab_size,Train_data.stoi["<pad>"],embedding_size,hidden_dim,num_layers,bidirectional,lin_layers,activation,outputs).to(device)
optimizer=optim.AdamW(model.parameters(),lr=lr)
model

Network(
  (Embedding): Embedding(30, 150, padding_idx=1)
  (LSTM): LSTM(150, 300, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (convert): Linear(in_features=1800, out_features=150, bias=True)
  (lin_layers): ModuleList(
    (0): Linear(in_features=150, out_features=100, bias=True)
  )
  (outputs): Linear(in_features=100, out_features=26, bias=True)
  (act): SELU()
)

In [130]:
for epoch in range(epochs):
    if epoch%200==0:
        x,y=MakeBatch(TrainX,TrainY,batchSize)
        x,y=[ele.to(device) for ele in x],y.to(device)
        logits,loss=model(x,y)
        print(f"for {epoch}/{epochs} Train loss {loss.cpu().detach().item()}")
        x,y=MakeBatch(ValX,ValY,batchSize)
        x,y=[ele.to(device) for ele in x],y.to(device)
        logits,loss=model(x,y)
        print(f"for {epoch}/{epochs} Val loss {loss.cpu().detach().item()}")
    optimizer.zero_grad()
    x,y=MakeBatch(TrainX,TrainY,batchSize)
    x,y=[ele.to(device) for ele in x],y.to(device)
    logits,loss=model(x,y)
    loss.backward()
    optimizer.step()
    

for 0/10000 Train loss 3.3772692680358887
for 0/10000 Val loss 3.361531972885132
for 200/10000 Train loss 2.9426233768463135
for 200/10000 Val loss 2.9666292667388916
for 400/10000 Train loss 2.8501980304718018
for 400/10000 Val loss 2.840642213821411
for 600/10000 Train loss 2.770334005355835
for 600/10000 Val loss 2.7945733070373535
for 800/10000 Train loss 2.7603089809417725
for 800/10000 Val loss 2.770395278930664
for 1000/10000 Train loss 2.7564454078674316
for 1000/10000 Val loss 2.7493200302124023
for 1200/10000 Train loss 2.7624056339263916
for 1200/10000 Val loss 2.7535722255706787
for 1400/10000 Train loss 2.741525411605835
for 1400/10000 Val loss 2.738529682159424
for 1600/10000 Train loss 2.7229456901550293
for 1600/10000 Val loss 2.743687391281128
for 1800/10000 Train loss 2.713451385498047
for 1800/10000 Val loss 2.6982333660125732
for 2000/10000 Train loss 2.7062134742736816
for 2000/10000 Val loss 2.7058022022247314
for 2200/10000 Train loss 2.7005512714385986
for 2200/

In [131]:
model

Network(
  (Embedding): Embedding(30, 150, padding_idx=1)
  (LSTM): LSTM(150, 300, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (convert): Linear(in_features=1800, out_features=150, bias=True)
  (lin_layers): ModuleList(
    (0): Linear(in_features=150, out_features=100, bias=True)
  )
  (outputs): Linear(in_features=100, out_features=26, bias=True)
  (act): SELU()
)

# Do not run the below models they were only put here to show work #

# Transformer's Encoder based model #

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[0,:x.size(1),:]
        return self.dropout(x)
def padding_mask(inputs,ins):
    mask=torch.ones_like(ins)
    for i,ele in enumerate(inputs):
        mask[i,:ele.shape[0]]=0
    return mask.bool()
class TransformersNetwork(nn.Module):
    def __init__(self,vocab_size,pad_idx,embedding_dim=300,num_head=5,num_stacks=2,layers=[100,50,26],outputs=26,activation=nn.ReLU()):
        super(TransformersNetwork,self).__init__()
        self.pad_idx=pad_idx
        self.Embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=self.pad_idx)
        self._init_weights(self.Embedding)
        self.pos_encoding=PositionalEncoding(embedding_dim)
        self.attentions=[nn.MultiheadAttention(embedding_dim,num_head,dropout=0.2,batch_first=True) for _ in range(num_stacks)]
        self.attentions=nn.ModuleList(self.attentions)
        self.lin_layers=[nn.Sequential(nn.Linear(embedding_dim,embedding_dim*4),nn.GELU(),nn.Linear(embedding_dim*4,embedding_dim)) for _ in range(num_stacks)]
        self.lin_layers=nn.ModuleList(self.lin_layers)
        for layer in self.lin_layers:
            for l in layer:
                self._init_weights(l)
        self.ln1=nn.LayerNorm(embedding_dim)
        self.ln2=nn.LayerNorm(embedding_dim)
        
        
        self.final_layers=[]
        self.final_layers.append(nn.Linear(embedding_dim,layers[0]))
        for i,j in zip(layers,layers[1:]):
            self.final_layers.append(nn.Linear(i,j))
        self.final_layers=nn.ModuleList(self.final_layers)
        self.outputs=nn.Linear(layers[-1],outputs)
        self._init_weights(self.outputs)
        for layer in self.final_layers:
            self._init_weights(layer)
        self.activation=activation
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_normal_(module.weight.data)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            nn.init.xavier_normal_(module.weight.data)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        
    def forward(self,inputs,outputs=None):
        ins=nn.utils.rnn.pad_sequence(inputs,padding_value=self.pad_idx,batch_first=True)
        embedding=self.Embedding(ins)
        pad_mask=padding_mask(inputs,ins)
        embedding_pos=self.pos_encoding(embedding)
        for attention,lin_layer in zip(self.attentions,self.lin_layers):
            atten_outs,_=attention(embedding_pos,embedding_pos,embedding_pos,key_padding_mask=pad_mask)
            atten_outs=self.ln1(atten_outs+embedding_pos)
            embedding_pos=self.ln2(atten_outs+lin_layer(atten_outs))
        
        averaged=embedding_pos.sum(dim=1)/(pad_mask==False).float().sum(dim=1,keepdims=True).to(device)
        for fin_layer in self.final_layers:
            averaged=self.activation(fin_layer(averaged))
        logits=self.outputs(averaged)
        if outputs is None:
            loss=None
        else:
            loss=nn.functional.cross_entropy(logits,outputs)
        
        return logits,loss   


# LSTM + Attention #

In [ ]:
class LSTM_Attention(nn.Module):
    def  __init__(self,vocab_size,pad_idx,embedding_dim=300,hidden_dim=50,num_layers=2,bidirectional=False,num_heads=10,num_stacks=3,lin_layers=[],activation=nn.ReLU(),outputs=26)-> None:
        super(Network,self).__init__()
        self.pad_idx=pad_idx
        self.Embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=self.pad_idx)
        self._init_weights(self.Embedding)
        self.LSTM=nn.LSTM(embedding_dim,hidden_dim,num_layers,bidirectional=bidirectional,dropout=0.4,batch_first=True)
        for layer in range(num_layers):
            for weight in self.LSTM._all_weights[layer]:
                if "weight" in weight:
                    nn.init.xavier_uniform_(getattr(self.LSTM,weight))
                if "bias" in weight:
                    nn.init.uniform_(getattr(self.LSTM,weight))
        
        #self.mha=nn.MultiheadAttention(embedding_dim,num_heads,batch_first=True,dropout=0.2)
        
        if(bidirectional):
        #    self.ln_norm=nn.LayerNorm(2*hidden_dim*num_layers)
            self.convert=nn.Linear(2*hidden_dim*num_layers,embedding_dim)
        else:
        #    self.ln_norm=nn.LayerNorm(hidden_dim*num_layers)
            self.convert=nn.Linear(hidden_dim*num_layers,embedding_dim)
        self.attentions=[nn.MultiheadAttention(embedding_dim,num_heads,dropout=0.2,batch_first=True) for _ in range(num_stacks)]
        self.attentions=nn.ModuleList(self.attentions)
        self.lin_layers=[nn.Sequential(nn.Linear(embedding_dim,embedding_dim*4),nn.GELU(),nn.Linear(embedding_dim*4,embedding_dim)) for _ in range(num_stacks)]
        self.lin_layers=nn.ModuleList(self.lin_layers)
        for layer in self.lin_layers:
            for l in layer:
                self._init_weights(l)
        self.fin_layers=[]
        self.fin_layers.append(nn.Linear(embedding_dim,lin_layers[0]))
        for i,j in zip(lin_layers,lin_layers[1:]):
            self.fin_layers.append(nn.Linear(i,j))
        self.fin_layers=nn.ModuleList(self.fin_layers)
        self.outputs=nn.Linear(lin_layers[-1],outputs)
        self._init_weights(self.outputs)
        self.act=activation
        self.ln1=nn.LayerNorm(embedding_dim)
        self.ln2=nn.LayerNorm(embedding_dim)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight.data)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            nn.init.kaiming_normal_(module.weight.data)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()

    def forward(self,x,avail,y=None):
        x=torch.nn.utils.rnn.pad_sequence(x,batch_first=True,padding_value=self.pad_idx)
        x=self.Embedding(x)
        o,(h,c)=self.LSTM(x)
        h=h.permute(1,0,2)
        x=h.reshape(h.shape[0],1,h.shape[1]*h.shape[2])
        avail_pad=torch.nn.utils.rnn.pad_sequence(avail,batch_first=True,padding_value=self.pad_idx)
        pad_mask=padding_mask(avail,avail_pad)
        avail=self.Embedding(avail_pad)
        x=self.convert(x)
        for attention,lin_layer in zip(self.attentions,self.lin_layers):
            atten_outs,_=attention(x,avail,avail)
            atten_outs=self.ln1(atten_outs+x)
            x=self.ln2(atten_outs+lin_layer(atten_outs))
        #atten_outs,atten_weights=self.mha(x,avail,avail)
        #print(atten_outs,atten_outs.shape)
        #x=self.act(self.convert(torch.squeeze(atten_outs,1)))
        x=torch.squeeze(x,1)
        #print(x.shape)
        for layer in self.fin_layers:
            x=layer(x)
            x=self.act(x)
        x=self.outputs(x)
        if y is None:
            loss=None
        else:
            loss=torch.nn.functional.cross_entropy(x,y)
        return x,loss
    def generate(self,x):
        logits,_=self(x)
        probs=torch.nn.functional.softmax(logits,dim=-1)
        idx=torch.multinomial(probs.cpu().detach(),1).item()
        return idx

# Bigram Model #  

In [97]:
class BiGramModel(nn.Module):
  def __init__(self,vocab_size,embed_size) -> None:
    super(BiGramModel,self).__init__()
    self.embed=nn.Embedding(vocab_size,embed_size)
    nn.init.kaiming_normal_(self.embed.weight)
    self.lin=nn.Linear(embed_size,vocab_size)
    nn.init.kaiming_normal_(self.lin.weight)
  def forward(self,idx,target=None):
    x=self.embed(idx)
    logits=self.lin(x)
    if target is None:
      loss=None
    else:
      B,T,C=logits.shape
      logs=logits.view(B*T,C)
      target=target.view(B*T)
      loss=nn.functional.cross_entropy(logs,target)
    return logits,loss
  def generate(self,idx):
    logits,_=self(idx)
    probs=nn.functional.softmax(logits[:,-1,:],dim=-1)
    idx_next=torch.multinomial(probs,num_samples=1)
    return idx_next

In [98]:
Data=open("words_250000_train.txt").readlines()
Data=[re.sub("\n","",word) for word in Data]
start="0"
end="1"
changed_data=[start+word+end for word in Data]
vocab=sorted(list(set("".join(changed_data))))
Bigram_stoi={a:i for i,a in enumerate(vocab)}
Bigram_itos={Bigram_stoi[a]:a for a in Bigram_stoi}
f="".join(changed_data)
block_size=10
batch_size=54
embed_size=100
vocab_size=len(vocab)

In [99]:
def encode(word,stoi=None):
    l=[]
    if stoi is not None:
        for i in word:
            l.append(stoi[i])
        return l
    for i in word:
        l.append(ord(i)-97)
    return l
def decode(tokens,itos):
    l=[]
    for i in tokens:
        l.append(itos[i])
    return "".join(l)

In [100]:
ForwardModel=BiGramModel(vocab_size,embed_size)
forwardoptimizer=optim.AdamW(ForwardModel.parameters(),lr=0.001)
BackwardModel=BiGramModel(vocab_size,embed_size)
backwardoptimizer=optim.AdamW(BackwardModel.parameters(),lr=0.001)

In [101]:
def makeBatchForward(text,block_size,batch_size):
  idx=torch.randint(0,len(text)-batch_size,(batch_size,))
  x=torch.tensor([encode(text[i:i+block_size],Bigram_stoi) for i in idx ])
  y=torch.tensor([encode(text[i+1:i+block_size+1],Bigram_stoi) for i in idx ])
  return x,y
def makeBatchBackward(text,block_size,batch_size):
  idx=torch.randint(0,len(text)-batch_size,(batch_size,))
  x=torch.tensor([encode(text[i+1:i+block_size+1],Bigram_stoi) for i in idx ])
  y=torch.tensor([encode(text[i:i+block_size],Bigram_stoi) for i in idx ])
  return x,y

In [102]:
#training the forwrd model
for i in range(10000):
  x,y=makeBatchForward(f,block_size,batch_size)
  logs,loss=ForwardModel(x,y)
  forwardoptimizer.zero_grad()
  loss.backward()
  forwardoptimizer.step()
print("Forward model loss:",loss)
#training the backward model
for i in range(10000):
  x,y=makeBatchBackward(f,block_size,batch_size)
  logs,loss=BackwardModel(x,y)
  backwardoptimizer.zero_grad()
  loss.backward()
  backwardoptimizer.step()
print("Backward model loss:",loss)

Forward model loss: tensor(2.3662, grad_fn=<NllLossBackward0>)
Backward model loss: tensor(2.3458, grad_fn=<NllLossBackward0>)


# Hangman game #

In [152]:
def encode(word,stoi=None):
    l=[]
    if stoi is not None:
        for i in word:
            l.append(stoi[i])
        return l
    for i in word:
        l.append(ord(i)-97)
    return l
def decode(tokens,itos):
    l=[]
    for i in tokens:
        l.append(itos[i])
    return "".join(l)

model=torch.load("./MODEL2.pt").to(device) # The model was locally saved as MODEL2.pt 
#gdrive link to model: https://drive.google.com/file/d/1xTy_1Ez3-3r7YkfEImOmTO7fksggIhx3/view?usp=sharing

print(model)
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
        self.current_dictionary = []
        #self.forwardmodel=ForwardModel
        #self.backwardmodel=BackwardModel
        self.model=model
        self.stoi=["<sos>","<eos>","<pad>","_"]+self.letters
        self.stoi.sort()
        self.stoi={ch:i for i,ch in enumerate(self.stoi)}
        self.itos={i:ch for i,ch in enumerate(self.letters)}
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        
        
        clean_word=re.sub(" ","",word)
        inputs=[self.stoi["<sos>"]]+encode(clean_word,self.stoi)+[self.stoi["<eos>"]]
        order=[]
        for ch in self.letters:
           if ch in self.guessed_letters or ch in clean_word:
               order.append("<pad>")
           else:
               order.append(ch)
        #left_out=[ch for ch in self.letters if ch not in clean_word and ch not in self.guessed_letters]
        inputs+=encode(order,self.stoi)
        letter=""
        logits,_=model(torch.tensor(inputs).view(1,-1).to(device))
        order=torch.argsort(logits[0],descending=True).tolist()
        logits=logits[0].tolist()
        for log in order:
           let=decode([log,],self.itos)
           if let not in self.guessed_letters:
               letter=let
               break
        
        return letter
        


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

Network(
  (Embedding): Embedding(30, 100, padding_idx=1)
  (LSTM): LSTM(100, 300, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (convert): Linear(in_features=1800, out_features=100, bias=True)
  (lin_layers): ModuleList()
  (outputs): Linear(in_features=100, out_features=26, bias=True)
  (act): SELU()
)


# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [153]:
api = HangmanAPI(access_token="", timeout=2000)


In [157]:
prev_runs,_,_,prev_successes=api.my_status()
prev_runs,prev_successes

(12750, 5695)

In [158]:
model

Network(
  (Embedding): Embedding(30, 100, padding_idx=1)
  (LSTM): LSTM(100, 300, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (convert): Linear(in_features=1800, out_features=100, bias=True)
  (lin_layers): ModuleList()
  (outputs): Linear(in_features=100, out_features=26, bias=True)
  (act): SELU()
)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [148]:
for i in range(1000):
    api.start_game(practice=1,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = (total_practice_successes - prev_successes)/ (total_practice_runs-prev_runs)
    print(practice_success_rate,(total_practice_successes - prev_successes),(total_practice_runs-prev_runs))
    time.sleep(0.5)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = (total_practice_successes - prev_successes)/ (total_practice_runs-prev_runs)
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


1.0 1 1
0.5 1 2
0.3333333333333333 1 3
0.5 2 4
0.6 3 5
0.6666666666666666 4 6
0.7142857142857143 5 7
0.75 6 8
0.7777777777777778 7 9
0.8 8 10
0.8181818181818182 9 11
0.75 9 12
0.6923076923076923 9 13
0.7142857142857143 10 14
0.6666666666666666 10 15
0.625 10 16
0.6470588235294118 11 17
0.6666666666666666 12 18
0.6842105263157895 13 19
0.65 13 20
0.6666666666666666 14 21
0.6363636363636364 14 22
0.6521739130434783 15 23
0.625 15 24
0.6 15 25
0.5769230769230769 15 26
0.5555555555555556 15 27
0.5357142857142857 15 28
0.5517241379310345 16 29
0.5666666666666667 17 30
0.5483870967741935 17 31
0.53125 17 32
0.5454545454545454 18 33
0.5588235294117647 19 34
0.5714285714285714 20 35
0.5555555555555556 20 36
0.5405405405405406 20 37
0.5263157894736842 20 38
0.5384615384615384 21 39
0.55 22 40
0.5365853658536586 22 41
0.5238095238095238 22 42
0.5348837209302325 23 43
0.5227272727272727 23 44
0.5333333333333333 24 45
0.5217391304347826 24 46
0.5319148936170213 25 47
0.5416666666666666 26 48
0.530

In [150]:
practice_success_rate,(total_practice_successes - prev_successes),(total_practice_runs-prev_runs)

(0.491, 491, 1000)

Its not >=50% but I hope it is good enough. Thank you 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [160]:
prev_runs,_,_,prev_successes=api.my_status()
prev_runs,prev_successes

(12760, 5701)

In [161]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [162]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.466


In [163]:
practice_success_rate,(total_practice_successes - prev_successes),(total_practice_runs-prev_runs)

(0.6, 0, 0)

In [5]:
model=torch.load("MODEL2.pt")
model

/home/senai2/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Network(
  (Embedding): Embedding(30, 100, padding_idx=1)
  (LSTM): LSTM(100, 300, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (convert): Linear(in_features=1800, out_features=100, bias=True)
  (lin_layers): ModuleList()
  (outputs): Linear(in_features=100, out_features=26, bias=True)
  (act): SELU()
)

In [6]:
model.eval()

Network(
  (Embedding): Embedding(30, 100, padding_idx=1)
  (LSTM): LSTM(100, 300, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (convert): Linear(in_features=1800, out_features=100, bias=True)
  (lin_layers): ModuleList()
  (outputs): Linear(in_features=100, out_features=26, bias=True)
  (act): SELU()
)